# Netflix Sentiment Analysis

# Imports

In [1]:
import numpy as np
import pandas as pd

# Functions

In [3]:
def check_for_nulls_dupes(df):
  print(f"The amount of NaN values: {df.isna().sum()}")
  print(f"The amount of Duplicated values: {df.duplicated().sum()}")

In [2]:
df = pd.read_csv("/Users/maukanmir/Downloads/netflix_reviews.csv",nrows=25000)
df

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,1772b40f-8cfd-4c56-a370-c944e5b5ce48,Gato Reviews,Utter trash. Been a customer for over 10 years...,1,0,8.114.0 build 19 50680,2024-05-23 22:35:45,8.114.0 build 19 50680
1,b6b64d5c-30f6-4bed-8d66-9b4582e72e3b,Quincy Alleyne,Good,5,0,8.115.2 build 6 50688,2024-05-23 22:34:47,8.115.2 build 6 50688
2,8be7d06b-f90e-4ca7-abf6-f908b40946e2,nez zuber,The only streaming service that beats Netflix ...,2,0,NaN,2024-05-23 22:33:52,NaN
3,72485684-9e73-4dca-a580-0a92b3a3a7e6,Elon Frett,Everything that's objectively good gets remove...,1,0,8.109.0 build 7 50647,2024-05-23 22:30:43,8.109.0 build 7 50647
4,348796c3-0056-4491-80f7-d6c38fcb2790,kwakhanya bambeni,Wish the owner be a trillioner because he/she ...,4,0,NaN,2024-05-23 22:30:27,NaN
...,...,...,...,...,...,...,...,...
24995,b6d3427b-1bd9-4767-86e4-6b69b4349908,avinaash Dey,Good but will be more if get a Hindi languages...,4,0,8.103.0 build 8 50614,2024-02-18 05:18:14,8.103.0 build 8 50614
24996,cb5480de-f2ab-44f0-97e5-c7e28f8b06ba,Dr. Gurmeet Singh Mankan,Nice and excellent performance for full entert...,5,1,8.103.0 build 8 50614,2024-02-15 02:39:01,8.103.0 build 8 50614
24997,4adfe2ba-b346-4afc-8db0-74d39e492a83,A Google user,Cool app but it is a bit frustrating for us fr...,2,0,7.53.3 build 31 34824,2020-04-27 21:12:37,7.53.3 build 31 34824
24998,3f852e4c-21df-4f3c-a122-e9d5f85a2e66,Biki Das,I am new to Netflix and u all know that withou...,3,0,NaN,2020-12-04 16:09:25,NaN


# Check for Nulls and Dupe Values

In [4]:
check_for_nulls_dupes(df)

The amount of NaN values: reviewId                   0
userName                   1
content                    2
score                      0
thumbsUpCount              0
reviewCreatedVersion    2549
at                         0
appVersion              2549
dtype: int64
The amount of Duplicated values: 287


In [5]:
df.dropna(inplace=True)

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.shape

(22224, 8)